<a href="https://colab.research.google.com/github/taaha3244/HuggingFace-NLP/blob/main/PEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
  torch==1.13.1 \
  torchdata==0.5.1 --quiet
!pip install \
  transformers==4.27.2 \
  datasets==2.11.0 \
  evaluate==0.4.0 \
  rouge_score==0.1.2 \
  loralib==0.1.1 \
  peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 38.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 req

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,GenerationConfig,TrainingArguments,Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [3]:
hugging_face_dataset="knkarthick/dialogsum"
dataset=load_dataset(hugging_face_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model_name='google/flan-t5-base'
original_model=AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)
tokenizer=AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
index=[200]
dash_line=''.join('' for i in range(100))
for i,index in enumerate(index):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""
Summarize the following conversation

{dialogue}

Summary:
  """

  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(
      original_model.generate(
          inputs['input_ids'],
          max_new_tokens=50
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print('Example',i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'Model Generation-Zero Shot:\n{output}\n')


Example 1

INPUT PROMPT:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

Model Generation-Zero Shot:
#Person1#: I'm thinking of upgrading my computer.



In [6]:
def tokenize_function(example):
  start_prompt='Summarize the following conversation. \n\n'
  end_prompt='\n\nSummary'
  prompt=[start_prompt + dialogue + end_prompt  for dialogue in example['dialogue']]
  example['input_ids']=tokenizer(prompt,padding='max_length',truncation=True,return_tensors='pt').input_ids
  example['labels']=tokenizer(example['summary'],padding='max_length',truncation=True,return_tensors='pt').input_ids
  return example



In [7]:
tokenized_datasets=dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets=tokenized_datasets.remove_columns(['id','topic','dialogue','summary'])

In [9]:
print(tokenized_datasets['train'][1])

print(tokenizer.decode(tokenized_datasets['train'][1]['input_ids']))


{'input_ids': [12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345, 13515, 536, 4663, 10, 8774, 8667, 5, 13156, 6, 149, 43, 25, 118, 58, 1713, 345, 13515, 357, 4663, 10, 8774, 707, 5, 2737, 7, 5, 1142, 1399, 2763, 25, 5, 11066, 63, 11, 27, 33, 270, 21, 112, 12956, 7, 5, 1713, 345, 13515, 536, 4663, 10, 4242, 168, 5, 1563, 31, 7, 217, 6, 1315, 12, 112, 24639, 1368, 6, 11066, 63, 65, 1204, 112, 6907, 32, 6, 2255, 17, 152, 302, 11, 216, 7768, 17, 159, 272, 6562, 5, 216, 19, 968, 767, 625, 6, 78, 3, 88, 19, 788, 21, 216, 7768, 17, 159, 71, 6, 16451, 102, 32, 226, 11, 1212, 9, 7, 965, 6562, 5, 1713, 345, 13515, 357, 4663, 10, 363, 81, 15612, 5303, 11, 283, 22028, 58, 1713, 345, 13515, 536, 4663, 10, 1548, 6, 27, 54, 163, 428, 376, 175, 21, 230, 6, 11, 227, 3, 9, 1158, 13, 1274, 27, 54, 24235, 8, 880, 5, 1713, 345, 13515, 357, 4663, 10, 6902, 6, 248, 5, 7582, 6, 27, 317, 27, 92, 164, 174, 3, 9, 2255, 17, 152, 302, 22205, 5, 2506, 97, 27, 530, 34, 47, 2087, 17310, 203, 977, 55, 1713, 345, 13515, 53

In [10]:
tokenized_datasets=tokenized_datasets.filter(lambda example, index: index%100==0,with_indices=True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [11]:
print(f"Shapes of the datasets")
print(f"Training : {tokenized_datasets['train'].shape}")
print(f"Validation : {tokenized_datasets['validation'].shape}")
print(f"Validation : {tokenized_datasets['test'].shape}.")

print(tokenized_datasets)

Shapes of the datasets
Training : (125, 2)
Validation : (5, 2)
Validation : (15, 2).
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


In [12]:
from peft import LoraConfig, get_peft_model,TaskType

lora_config=LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q','v'],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [15]:
from peft import get_peft_model

model = get_peft_model(original_model, lora_config)
model.print_trainable_parameters()
"output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282"

trainable params: 1769472 || all params: 249347328 || trainable%: 0.7096414524241463


'output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282'

In [ ]:
output_dir='taaha/flan-t5'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-3,
    num_train_epochs=1,
    auto_find_batch_size=True,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    max_steps=1,
    logging_steps=1
)

peft_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

peft_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
